<h1>Dynammic Programming</h1>

# 0. Policy Evaluation (Prediction), output $V \approx V_{\pi}(s)$

## Enviroment
The environment is like, 

> 0, 1, 2, 3

> 4, 5, 6, 7

> 8, 9, 10, 11

> 12, 13, 14, 15

## State
For state $s \in \{0, 1, ..., 15\}$, with 0 and 15 are terminal states .

## Action

The agent can go as one of the actions in $\{up, down, left, right\}$.

## Transition Probability

The transition probability of the agent goes from $s$ to $s'$ will be 1 if $s'$ is the next state of $s$ with action in $\mathcal{S}$ or $s$ is in the side of the matrix, otherwise 0.

## Reward

If the next state $s'$ is the terminal state, the reward is 0, otherwise -1.

## Policy Evaluation

+ Initialize

> $V(s) = 0, \forall s \in \mathcal{S}^{+}$

+ Repeat

> $\Delta \leftarrow 0$

> For each $s \in \mathcal{S}$:

> > $v \leftarrow V(s)$

> > $V(s) \leftarrow \sum_{a}\pi(a|s) \sum_{s', r}p(s', r|s, a)[r + \gamma V(s')]$

> > $\Delta \leftarrow \max(\Delta, |v - V(s)|)$

+ until $\Delta < \theta$

In [1]:
import numpy as np 

# environment
env = np.array([[0, 1, 2, 3], 
                [4, 5, 6, 7], 
                [8, 9, 10, 11], 
               [12, 13, 14, 15]])

In [2]:
# state 
s = np.array(range(16))

In [3]:
# transition probability
n_s = 16
n_a = 4 #actions, 0: up, 1: down, 2: left , 3: right
p = np.zeros((n_s, n_a, n_s)) # s, a, s'

prob = 1. / 4 #prob is equal for all actions

#up 
for s in range(4):
    p[s][0][s] = prob
for s in range(4, 16):
    p[s][0][s-4]= prob
    
#down
for s in range(12, 16):
    p[s][1][s] = prob
for s in range(0, 12):
    p[s][1][s+4] = prob
    
# left 
for s in range(0, 16, 4):
    p[s][2][s] = prob
for s in range(1,16):
    if s % 4 != 0:
        p[s][2][s-1] = prob

# right
for s in range(3, 16, 4):
    p[s][3][s] = prob
for s in range(0, 16):
    if s % 4 != 3:
        p[s][3][s+1] = prob
        
# terminal state
for a in range(4):
    for s1 in range(16):
        p[0][a][s1] = 0
        p[15][a][s1] = 0

In [4]:
# Order for args: 
# s, a, s1, gamma, p, pi, V 

# reward function
def reward_func(s, a, s1):
    if s1 == 0 or s1 == 15:
        return 0
    return -1

In [5]:
def expect_value(s, gamma, p, V):
    """
    s: current state
    gamma: discount factor
    V: value function
    p: transition probability, s, a, s1
    """
    
    # pi(a|s) = 1 / n_a, for all actions
    
    e_v = 0
    for s1 in range(16):
        for a in range(4):
            #pi_s_a = 1. / 4
            p_s_a_s1 = p[s][a][s1]
            r = reward_func(s, a, s1)
            e_v += p_s_a_s1 * (r + gamma * V[s1])

    return e_v

In [9]:
def evaluate_value(gamma, p, V):
    delta = 0
    for s in range(16):
        v = V[s]
        V[s] = expect_value(s, gamma, p, V)
        delta = max(delta, abs(v - V[s]))
    return V, delta

In [12]:
# Initialize
V = np.zeros(16)
gamma = 1.0

def policy_evaluation(gamma, p, V):
    delta = 1e10
    epsilon = 1e-4
    while delta > epsilon:
        V, delta = evaluate_value(gamma, p, V)
    return V

In [13]:
V = policy_evaluation(gamma, p, V)

In [14]:
V

array([  0.        , -12.99934883, -18.99906386, -20.9989696 ,
       -12.99934883, -16.99920093, -18.99913239, -18.99914232,
       -18.99906386, -18.99913239, -16.9992679 , -12.9994534 ,
       -20.9989696 , -18.99914232, -12.9994534 ,   0.        ])

# 1. Policy Iteration, output $V \approx v_{*}$ and $\pi \approx \pi_{*}$

+ Initialization

> $V(s) \in \mathbb{R}, \pi(s) \in \mathcal{A}(s)$ arbitrarily

+ Policy Evaluation

+ Policy Improvement

> $policy\_stable \leftarrow true$

> For each $s \in \mathcal{S}$:

> > $old\_action \leftarrow \pi(s)$

> > $\pi(s) \leftarrow \arg\max_{a}\sum_{s', r}p(s', r|s, a)[r + \gamma V(s')]$

> > If $old\_action \ne \pi(s)$, then $policy\_stable \leftarrow false$

> If $policy\_stable = true$, then stop, else goto 2.

In [35]:
def expect_pi(s, gamma, p, V):
    max_a = -1
    max_r = -1e10
    
    for a in range(4):
        e_v = 0 # expect_value
        for s1 in range(16):
            p_s_a_s1 = p[s][a][s1]
            r = reward_func(s, a, s1)
            e_v += p_s_a_s1 * (r + gamma * V[s1])
        
        if e_v > max_r:
            max_r = e_v
            max_a = a
    
    return max_a


def policy_improvement(gamma, p, pi, V):
    policy_stable = True
    
    for s in range(16):
        old_a = pi[s]

        pi[s] = expect_pi(s, gamma, p, V)

        if old_a != pi[s]:
            policy_stable = False

    return V, pi, policy_stable        

In [36]:
# Initialize
n_s = 16
n_a = 4
V = np.zeros(n_s)
pi = np.zeros(n_s) 

policy_stable = False

# policy iteration
while policy_stable == False:
    V = policy_evaluation(gamma, p, V)
    V, pi, policy_stable = policy_improvement(gamma, p, pi, V)


print(V)
print(pi)

[  0.         -12.99940341 -18.99914232 -20.99905596 -12.99940341
 -16.9992679  -18.99920511 -18.9992142  -18.99914232 -18.99920511
 -16.99932926 -12.99949921 -20.99905596 -18.9992142  -12.99949921
   0.        ]
[0. 2. 2. 2. 0. 0. 2. 1. 0. 0. 3. 1. 0. 3. 3. 0.]
